In [1]:
import os
import sys
print(os.getcwd())
sys.path.append("../src")
sys.path.append("../src/data/")
sys.path.append("../src/models/")

import tensorflow as tf
from dataloader import *
from network import *
from loss import *
from config import *
from utils import *
import logging
import os

from models.loss import bceDiceLoss, diceCoef
sys.path.append("/work/08940/joshuaeh/maverick2/miniconda3/lib/:/work/08940/joshuaeh/maverick2/miniconda3/lib/:/work/08940/joshuaeh/maverick2/miniconda3/lib/:/work/08940/joshuaeh/maverick2/miniconda3/envs/tf/lib/:/work/08940/joshuaeh/maverick2/miniconda3/envs/tf/lib/:/work/08940/joshuaeh/maverick2/miniconda3/lib/:/work/08940/joshuaeh/maverick2/miniconda3/envs/tf/lib/:/work/08940/joshuaeh/maverick2/miniconda3/envs/tf/lib/")

/work/08940/joshuaeh/maverick2/projects/ACLNet/src


/work/08940/joshuaeh/maverick2/miniconda3/envs/tf/lib/python3.10/site-packages/albumentations/augmentations/transforms.py:1613: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/work/08940/joshuaeh/maverick2/miniconda3/envs/tf/lib/python3.10/site-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2022-09-08 09:13:31.507753: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /work/08940/joshuaeh/maverick2/miniconda3/envs/tf/lib/python3.10/site-packages/cv2/../../lib64:/opt/intel/compilers_and_libraries_2018.2.199/linux/mpi/intel64/lib:/opt/intel/debugger_2018/libipt/intel64/lib:/opt/intel/debugger_2018/iga/lib:/opt/intel/compilers_and_libraries_2018.2.199/linux/daal/../tbb/lib/intel64_lin/gcc4.4:/opt/intel/compilers_and_libraries_2018.2.199/linux/daal/lib/intel64_lin:/opt/intel/compilers_and_libraries_2018.2.199/linux/tbb/lib/intel64/gcc4.7:/opt/intel/compilers_and_libraries_2018.2.199/linux/mkl/lib/intel64_lin:/opt/intel/compilers_and_libraries_2018.2.199/linux/compiler/lib/intel64_lin:/opt/intel/compilers_and_libraries_2018.2.199/linux/ipp/lib/intel64:/opt/intel/compilers_and_libraries_2018.2.199/linux/compiler/lib/intel

In [14]:
logging.basicConfig(level = logging.INFO, filename = os.path.join(LOG_DIR, 'app.log'), format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', filemode='w')

sys.stdout = LoggerWriter(logging.info)
sys.stderr = LoggerWriter(logging.error)

logger = logging.getLogger(__name__)

logger.info("[Info] Getting DataLoader")
trainGen, testGen = getDataLoader(batch_size=BATCH_SIZE)

logger.info("[Info] Creating Network") 
model = tf.keras.models.load_model("../weights/ACLNet_Best.h5", custom_objects={"bceDiceLoss":bceDiceLoss, "diceCoef":diceCoef})

model.compile(optimizer = tf.keras.optimizers.Adam(INITIAL_LEARNING_RATE), loss = bceDiceLoss, metrics = [diceCoef])
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(os.path.join(WEIGHTS_DIR, 'ACLNet_Best.h5'), monitor = 'val_diceCoef', mode="max", verbose = 1, save_best_only = True, save_weights_only = False),
    tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_diceCoef', mode="max", factor = 0.1, patience = 20, min_lr = 0.00001)
]
model.fit(trainGen, validation_data = testGen, epochs = EPOCHS, verbose = 1, callbacks = callbacks)
logger.info("[Info] Training Finished")